In [ ]:
!pip install transformers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install gradio

# **test1**
ê test1 đúng hơn nha

In [ ]:
import tensorflow as tf

# Tối ưu TensorFlow để sử dụng GPU trước khi khởi tạo mô hình
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except RuntimeError as e:
        print(f"Không thể thiết lập set_memory_growth: {e}")


import gradio as gr
import re
import numpy as np
from collections import Counter
from nltk.util import ngrams
from tensorflow.keras.models import load_model
from transformers import pipeline
import string
import tensorflow as tf

# Tải mô hình đầu tiên và đảm bảo chạy trên GPU nếu có
model = load_model('/content/drive/MyDrive/Final/spelling_epoch_03.keras')
model.make_predict_function()

# Các biến số
NGRAM = 2
MAXLEN = 40  # Kích thước đầu vào của mô hình
alphabet = ['\x00', ' ', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'í', 'ì', 'ỉ', 'ĩ', 'ị', 'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'đ', 'Á', 'À', 'Ả', 'Ã', 'Ạ', 'Â', 'Ấ', 'Ầ', 'Ẩ', 'Ẫ', 'Ậ', 'Ă', 'Ắ', 'Ằ', 'Ẳ', 'Ẵ', 'Ặ', 'Ó', 'Ò', 'Ỏ', 'Õ', 'Ọ', 'Ô', 'Ố', 'Ồ', 'Ổ', 'Ỗ', 'Ộ', 'Ơ', 'Ớ', 'Ờ', 'Ở', 'Ỡ', 'Ợ', 'É', 'È', 'Ẻ', 'Ẽ', 'Ẹ', 'Ê', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ư', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Í', 'Ì', 'Ỉ', 'Ĩ', 'Ị', 'Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'Đ']
accepted_char = list(string.digits + ''.join(alphabet))

# Định nghĩa hàm call từ mô hình đầu tiên
def call(sentence):
    def extract_phrases(text):
        pattern = r'\w[\w ]*|\s\W+|\W+'
        return re.findall(pattern, text)

    def encoder_data(text, maxlen=MAXLEN):
        x = np.zeros((maxlen, len(alphabet)))
        for i, c in enumerate(text[:maxlen]):
            if c in alphabet:
                x[i, alphabet.index(c)] = 1
        if i < maxlen - 1:
            for j in range(i + 1, maxlen):
                x[j, 0] = 1
        return x

    def decoder_data(x):
        x = x.argmax(axis=-1)
        return ''.join(alphabet[i] for i in x)

    def nltk_ngrams(words, n=2):
        return ngrams(words.split(), n)

    def batch_predict(ngrams_batch):
        batch_input = np.array([encoder_data(' '.join(ngram)) for ngram in ngrams_batch])
        preds = model.predict(batch_input, verbose=0)
        return [decoder_data(pred).strip('\x00') for pred in preds]

    def separate_words(text):
        return re.sub(r'(?<!^)(?=[A-Z])', ' ', text)

    def correct(sentence):
        sentence = separate_words(sentence)

        for i in sentence:
            if i not in accepted_char:
                sentence = sentence.replace(i, " ")

        ngrams_list = list(nltk_ngrams(sentence, n=NGRAM))
        guessed_ngrams = batch_predict(ngrams_list)  # Gọi batch_predict thay vì từng n-gram

        candidates = [Counter() for _ in range(len(guessed_ngrams) + NGRAM - 1)]
        for nid, ngram in enumerate(guessed_ngrams):
            for wid, word in enumerate(re.split(' +', ngram)):
                candidates[nid + wid].update([word])

        if not candidates or all(len(c) == 0 for c in candidates):
            return "Không có từ nào để sửa."

        output = ' '.join(c.most_common(1)[0][0] for c in candidates if c)
        return output

    return correct(sentence)

# Hàm để xử lý đầu vào và đầu ra của Gradio
def correct_sentence(sentence):
    # Gọi mô hình đầu tiên
    first_guess = call(sentence)

    # Sử dụng mô hình transformers đã lưu và chạy trên GPU nếu có
    corrector = pipeline("text2text-generation", model="/content/drive/MyDrive/final2/datasettest1", tokenizer="/content/drive/MyDrive/final2/datasettest1", device=0)  # device=0 để chạy trên GPU

    # Chuyển kết quả từ mô hình đầu tiên sang mô hình transformers
    MAX_LENGTH = 512
    predictions = corrector(first_guess, max_length=MAX_LENGTH)

    # Hiển thị kết quả cuối cùng sau khi mô hình transformers chạy
    final_output = predictions[0]['generated_text']

    return final_output

# Tạo giao diện Gradio
iface = gr.Interface(
    fn=correct_sentence,
    inputs="text",
    outputs="text",
    title="Vietnamese Spell Correction",
    description="Nhập một câu có thể có lỗi chính tả và nhấn 'Submit' để nhận kết quả sửa lỗi từ mô hình transformers."
)

# Chạy giao diện
iface.launch(share=True)


In [ ]:
from transformers import pipeline

corrector = pipeline("text2text-generation", model="bmd1905/vietnamese-correction-v2")


In [ ]:
MAX_LENGTH = 512
texts = ["CỘNG HÒA XÃ HỘI CHỦ NGỈA VIỆT NAM Độc lập - Tự do - Hạn phúc CĂN CUỚC CÔNG DÂN 030092000101 Họ vầ tên NGYỄN VĂN HẬU Ngày, thág, năm xinh: 12/04/1992 Giới tínhh Nam Quốc tịch Vịêt Nam Quê quán Ngọc Kỳ, Tứ Kì, Hải Đương Nơi thường trú: Ngoc Kỳ, Tứ Kỳ, Hải Đuong Có giá trị đén: 12/04/2032"
]
# Batch prediction
predictions = corrector(texts, max_length=MAX_LENGTH)

# Print predictions
for text, pred in zip(texts, predictions):
    print("- " + pred['generated_text'])

In [ ]:
!pip install gradio_client

In [ ]:
!pip install pyvi

# **# test2**

In [ ]:
import tensorflow as tf

# Tối ưu TensorFlow để sử dụng GPU trước khi khởi tạo mô hình
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except RuntimeError as e:
        print(f"Không thể thiết lập set_memory_growth: {e}")

import gradio as gr
import re
import numpy as np
from collections import Counter
from nltk.util import ngrams
from tensorflow.keras.models import load_model
from transformers import pipeline
import string

# Tải mô hình đầu tiên và đảm bảo chạy trên GPU nếu có
model = load_model('/content/drive/MyDrive/Final/spelling_epoch_03.keras')
model.make_predict_function()

# Các biến số
NGRAM = 2
MAXLEN = 40  # Kích thước đầu vào của mô hình
alphabet = ['\x00', ' ', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'í', 'ì', 'ỉ', 'ĩ', 'ị', 'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'đ', 'Á', 'À', 'Ả', 'Ã', 'Ạ', 'Â', 'Ấ', 'Ầ', 'Ẩ', 'Ẫ', 'Ậ', 'Ă', 'Ắ', 'Ằ', 'Ẳ', 'Ẵ', 'Ặ', 'Ó', 'Ò', 'Ỏ', 'Õ', 'Ọ', 'Ô', 'Ố', 'Ồ', 'Ổ', 'Ỗ', 'Ộ', 'Ơ', 'Ớ', 'Ờ', 'Ở', 'Ỡ', 'Ợ', 'É', 'È', 'Ẻ', 'Ẽ', 'Ẹ', 'Ê', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ư', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Í', 'Ì', 'Ỉ', 'Ĩ', 'Ị', 'Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'Đ']
accepted_char = list(string.digits + ''.join(alphabet))

# Định nghĩa hàm call từ mô hình đầu tiên
def call(sentence):
    def extract_phrases(text):
        pattern = r'\w[\w ]*|\s\W+|\W+'
        return re.findall(pattern, text)

    def encoder_data(text, maxlen=MAXLEN):
        x = np.zeros((maxlen, len(alphabet)))
        for i, c in enumerate(text[:maxlen]):
            if c in alphabet:
                x[i, alphabet.index(c)] = 1
        if i < maxlen - 1:
            for j in range(i + 1, maxlen):
                x[j, 0] = 1
        return x

    def decoder_data(x):
        x = x.argmax(axis=-1)
        return ''.join(alphabet[i] for i in x)

    def nltk_ngrams(words, n=2):
        return ngrams(words.split(), n)

    def batch_predict(ngrams_batch):
        batch_input = np.array([encoder_data(' '.join(ngram)) for ngram in ngrams_batch])
        preds = model.predict(batch_input, verbose=0)
        return [decoder_data(pred).strip('\x00') for pred in preds]

    def separate_words(text):
        return re.sub(r'(?<!^)(?=[A-Z])', ' ', text)

    def correct(sentence):
        sentence = separate_words(sentence)

        for i in sentence:
            if i not in accepted_char:
                sentence = sentence.replace(i, " ")

        ngrams_list = list(nltk_ngrams(sentence, n=NGRAM))
        guessed_ngrams = batch_predict(ngrams_list)

        candidates = [Counter() for _ in range(len(guessed_ngrams) + NGRAM - 1)]
        for nid, ngram in enumerate(guessed_ngrams):
            for wid, word in enumerate(re.split(' +', ngram)):
                candidates[nid + wid].update([word])

        if not candidates or all(len(c) == 0 for c in candidates):
            return "Không có từ nào để sửa."

        output = ' '.join(c.most_common(1)[0][0] for c in candidates if c)
        return output

    return correct(sentence)

# Hàm để xử lý đầu vào và đầu ra của Gradio
def correct_sentence(sentence):
    # Chuyển tất cả các ký tự thành chữ thường
    sentence = sentence.lower()

    # Gọi mô hình đầu tiên
    first_guess = call(sentence)

    # Sử dụng mô hình transformers đã lưu và chạy trên GPU nếu có
    corrector = pipeline("text2text-generation", model="/content/drive/MyDrive/final2/datasettest1", tokenizer="/content/drive/MyDrive/final2/datasettest1", device=0)

    # Chuyển kết quả từ mô hình đầu tiên sang mô hình transformers
    MAX_LENGTH = 512
    predictions = corrector(first_guess, max_length=MAX_LENGTH)

    # Hiển thị kết quả cuối cùng sau khi mô hình transformers chạy
    final_output = predictions[0]['generated_text']

    return final_output

# Tạo giao diện Gradio
iface = gr.Interface(
    fn=correct_sentence,
    inputs="text",
    outputs="text",
    title="Vietnamese Spell Correction",
    description="Nhập một câu có thể có lỗi chính tả và nhấn 'Submit' để nhận kết quả sửa lỗi từ mô hình transformers."
)

# Chạy giao diện
iface.launch(share=True)


In [ ]:
import tensorflow as tf
import gradio as gr
import numpy as np
import re
from collections import Counter
from nltk.util import ngrams
from tensorflow.keras.models import load_model
from transformers import pipeline
import string
import time

# Giải phóng bộ nhớ TensorFlow trước khi tải mô hình
tf.keras.backend.clear_session()

# Chỉ tải mô hình một lần để tiết kiệm tài nguyên
model = load_model('/content/drive/MyDrive/Final/my_model_weight.keras')
model.make_predict_function()
# Các biến số
NGRAM = 2
MAXLEN = 40  # Kích thước đầu vào của mô hình
alphabet = ['\x00', ' ', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
            'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
            '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
            'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ',
            'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ',
            'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ',
            'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự',
            'í', 'ì', 'ỉ', 'ĩ', 'ị', 'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ',
            'đ', 'Á', 'À', 'Ả', 'Ã', 'Ạ', 'Â', 'Ấ', 'Ầ', 'Ẩ', 'Ẫ', 'Ậ', 'Ă', 'Ắ', 'Ằ', 'Ẳ', 'Ẵ', 'Ặ',
            'Ó', 'Ò', 'Ỏ', 'Õ', 'Ọ', 'Ô', 'Ố', 'Ồ', 'Ổ', 'Ỗ', 'Ộ', 'Ơ', 'Ớ', 'Ờ', 'Ở', 'Ỡ', 'Ợ',
            'É', 'È', 'Ẻ', 'Ẽ', 'Ẹ', 'Ê', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ',
            'Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ư', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự',
            'Í', 'Ì', 'Ỉ', 'Ĩ', 'Ị', 'Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ',
            'Đ']
accepted_char = list(string.digits + ''.join(alphabet))

# Định nghĩa hàm call từ mô hình đầu tiên
def call(sentence):
    def extract_phrases(text):
        pattern = r'\w[\w ]*|\s\W+|\W+'
        return re.findall(pattern, text)

    def encoder_data(text, maxlen=MAXLEN):
        x = np.zeros((maxlen, len(alphabet)))
        for i, c in enumerate(text[:maxlen]):
            if c in alphabet:
                x[i, alphabet.index(c)] = 1
        if i < maxlen - 1:
            for j in range(i + 1, maxlen):
                x[j, 0] = 1
        return x

    def decoder_data(x):
        x = x.argmax(axis=-1)
        return ''.join(alphabet[i] for i in x)

    def nltk_ngrams(words, n=2):
        return ngrams(words.split(), n)

    def batch_predict(ngrams_batch):
        batch_input = np.array([encoder_data(' '.join(ngram)) for ngram in ngrams_batch])
        preds = model.predict(batch_input, verbose=0)
        return [decoder_data(pred).strip('\x00') for pred in preds]

    def separate_words(text):
        return re.sub(r'(?<!^)(?=[A-Z])', ' ', text)

    def correct(sentence):
        sentence = separate_words(sentence)

        for i in sentence:
            if i not in accepted_char:
                sentence = sentence.replace(i, " ")

        ngrams_list = list(nltk_ngrams(sentence, n=NGRAM))
        guessed_ngrams = batch_predict(ngrams_list)  # Gọi batch_predict thay vì từng n-gram

        candidates = [Counter() for _ in range(len(guessed_ngrams) + NGRAM - 1)]
        for nid, ngram in enumerate(guessed_ngrams):
            for wid, word in enumerate(re.split(' +', ngram)):
                candidates[nid + wid].update([word])

        if not candidates or all(len(c) == 0 for c in candidates):
            return "Không có từ nào để sửa."

        output = ' '.join(c.most_common(1)[0][0] for c in candidates if c)
        return output

    return correct(sentence)

# Hàm để xử lý đầu vào và đầu ra của Gradio
def correct_sentence(sentence):
    # Thời gian bắt đầu
    start_time = time.time()

    # Gọi mô hình đầu tiên
    first_guess = call(sentence)

    # Sử dụng mô hình transformers đã lưu và chạy trên CPU
    corrector = pipeline("text2text-generation", model="/content/drive/MyDrive/Final/tanphuc_model", tokenizer="/content/drive/MyDrive/Final/tanphuc_model", device=-1)  # device=-1 để chạy trên CPU

    # Chuyển kết quả từ mô hình đầu tiên sang mô hình transformers
    MAX_LENGTH = 512
    predictions = corrector(first_guess, max_length=MAX_LENGTH)

    # Hiển thị kết quả cuối cùng
    final_output = predictions[0]['generated_text']

    # Tính thời gian chạy
    run_time = time.time() - start_time

    # Tính số lượng từ đã sửa và hiển thị từ đã sửa
    words_original = sentence.split()
    words_corrected = final_output.split()

    corrected_words = []
    for original, corrected in zip(words_original, words_corrected):
        if original != corrected:
            corrected_words.append(f"{original} -> {corrected}")

    num_corrected = len(corrected_words)

    return final_output, run_time, num_corrected, corrected_words

# Tạo giao diện Gradio
iface = gr.Interface(
    fn=correct_sentence,
    inputs="text",
    outputs=["text", "text", "text", "text"],  # Thêm đầu ra cho thời gian và số từ đã sửa
    title="Vietnamese Spell Correction",
    description="Nhập một câu có thể có lỗi chính tả và nhấn 'Submit' để nhận kết quả sửa lỗi từ mô hình transformers."
)

# Chạy giao diện
iface.launch(share=True)
